# Connect to database

In [4]:
from query_database import QueryDatabase

df = QueryDatabase("SELECT * FROM supplier_invoice.invoice_202408;").df
df.head()

,Förvaltning,Leverantör,Organisationsnummer,Verifikationsnummer,Konto,Kontotext,Belopp exkl moms
0,Stadsmiljönämnden,TRACK TEC GMBH,106/5727/0626,4001291513,4101,Inköp anläggnings och underhållsmaterial,"9 835 315,00"
1,Kretslopp och Vatten,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,5601378982,6185,Anläggningsentreprenad,"870,00"
2,Kretslopp och Vatten,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,5601377374,6185,Anläggningsentreprenad,"870,00"
3,Kretslopp och Vatten,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,5601378519,6185,Anläggningsentreprenad,"870,00"
4,Exploateringsnämnden,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,2001226894,7641,Diverse skatter och offentliga avgifter,"1 000,00"


In [5]:
df.shape

(92989, 7)

## Clean cost column

In [10]:
df.loc[:, "Belopp exkl moms"] = df["Belopp exkl moms"].str[:-3].str.replace(" ","").astype(int)
df.head()

,Förvaltning,Leverantör,Organisationsnummer,Verifikationsnummer,Konto,Kontotext,Belopp exkl moms
0,Stadsmiljönämnden,TRACK TEC GMBH,106/5727/0626,4001291513,4101,Inköp anläggnings och underhållsmaterial,9835315
1,Kretslopp och Vatten,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,5601378982,6185,Anläggningsentreprenad,870
2,Kretslopp och Vatten,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,5601377374,6185,Anläggningsentreprenad,870
3,Kretslopp och Vatten,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,5601378519,6185,Anläggningsentreprenad,870
4,Exploateringsnämnden,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,2001226894,7641,Diverse skatter och offentliga avgifter,1000


In [12]:
from database import Database
from constants import DATABASE_PATH

with Database(DATABASE_PATH) as db:
    db.connection.register('df', df)

    db.query("CREATE SCHEMA IF NOT EXISTS refined_layer;")
    db.query(f"""
             CREATE TABLE IF NOT EXISTS refined_layer.invoice_202408
             AS SELECT * FROM df;
             """)

In [14]:
QueryDatabase("SELECT * FROM refined_layer.invoice_202408;").df.head()

,Förvaltning,Leverantör,Organisationsnummer,Verifikationsnummer,Konto,Kontotext,Belopp exkl moms
0,Stadsmiljönämnden,TRACK TEC GMBH,106/5727/0626,4001291513,4101,Inköp anläggnings och underhållsmaterial,9835315
1,Kretslopp och Vatten,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,5601378982,6185,Anläggningsentreprenad,870
2,Kretslopp och Vatten,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,5601377374,6185,Anläggningsentreprenad,870
3,Kretslopp och Vatten,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,5601378519,6185,Anläggningsentreprenad,870
4,Exploateringsnämnden,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,2001226894,7641,Diverse skatter och offentliga avgifter,1000


In [19]:
QueryDatabase("""
              SELECT
                leverantör,
                SUM("Belopp exkl moms") AS "Summa (SEK)"
              FROM 
                refined_layer.invoice_202408
              GROUP BY
                leverantör
              ORDER BY
                "Summa (SEK)"
              DESC LIMIT 10;
              """).df

,Leverantör,Summa (SEK)
0,GÖTEBORGS STADS LEASING AB,62433766.0
1,KPA PENSIONSSERVICE AB,59741750.0
2,RÄDDNINGSTJÄNSTEN STORGÖTEBORG,55320824.0
3,PURAC AB,48603208.0
4,RENOVA,34435107.0
5,VÄSTTRAFIK AB,28616573.0
6,MARK OCH ENERGIBYGGARNA,24775068.0
7,ATTENDO SVERIGE AB,23593435.0
8,MARTIN&SERVERA RESTAURANGHANDEL,23323653.0
9,GRYAAB AB,22961946.0
